# Docking

This notebook takes two files as input: a .pdb file for the receptor (aptamer) and a .sdf or .pdb file for the ligand. It is necessary to indicate the whole path for these files at the "Locate your files" step.

Both files will be used to reduce the molecules and obtain .pdbqt files, which will serve as input for AutoDock Vina to perform docking.

Before performing docking, it is necessary to create a parameters file, "param_file", containing the values of the parameters AutoDock Vina needs: xc, yc, zc, xs, ys, and zc. It is necessary to provide a unique name for this file for every experiment by writing it on the variable "param_file", and to provide the specific value for each parameter, which will value for the different molecules to dock.

As an output, this dock provides an .pdbqt file, which represents the results of the molecular docking simulation. They can be visualized using molecular visualization programs, such as Chimera or PyMOL.

## 1. Create necessary directories


In [21]:
%%bash

mkdir -p PDBQT_files
mkdir -p Original_files
mkdir -p Vina_results
mkdir -p Vina_params

## 2. Locate your files (pdb or sdf) in the Original_files directory

In [25]:
Aptamer_path="../../data/3dRNA_DNA_PFOA2.pdb"
Ligand_path="../../data/PFOA_sdf2.sdf"

Aptamer_file="3dRNA_DNA_PFOA2.pdb"
Ligand_file="PFOA_sdf2.sdf"

Aptamer_name="3dRNA_DNA_PFOA2"
Ligand_name="PFOA_sdf2"

In [26]:
%%bash -s "$Aptamer_path" "$Ligand_path"

cp $1 Original_files/
cp $2 Original_files/


## 3.Make sure both receptor and ligand are reduced, and create PDBQT files

In [27]:
# Check if Aptamer_name ends with ".sdf"
if Ligand_file.endswith(".sdf"):
    format_ligand = "sdf"
# Check if Aptamer_name ends with ".pdb"
elif Ligand_file.endswith(".pdb"):
    format_ligand = "pdb"
# If neither ".sdf" nor ".pdb" is found at the end, you can provide a default value
else:
    print("Please, ensure that your ligand is in either .sdf or .pdb format")


In [29]:
%%bash -s "$Aptamer_name" "$Ligand_name" "$format_ligand"

# Reduce the aptamer
obabel -i pdb Original_files/$1.pdb -o pdb -h > Original_files/$1_H.pdb
# Get the pdbqt for the aptamer
obabel Original_files/$1_H.pdb -O PDBQT_files/$1.pdbqt -p 7.4 -xr


# Reduce the ligand
# Check if $3 equals "sdf"
if [ "$3" = "sdf" ]; then
    obabel -i sdf Original_files/$2.sdf -o sdf -h > Original_files/$2_H.sdf
    obabel -isdf Original_files/$2_H.sdf -opdbqt -O PDBQT_files/$2.pdbqt
# Check if $3 equals "pdb"
elif [ "$3" = "pdb" ]; then
    obabel -i pdb Original_files/$2.pdb -o pdb -h > Original_files/$2_H.pdb
    obabel Original_files/$2_H.pdb -O PDBQT_files/$2.pdbqt -p 7.4

else
    # Handle other cases or provide an error message
    echo "Invalid value for \$3: $3"
    exit 1
fi


1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


## 4.Run AutodockVina

In [32]:
%%bash -s "$Aptamer_name" "$Ligand_name"

# Create parameters' file

param_file="Ap2_params.txt"
if [ ! -f Vina_params/"$param_file" ]; then
  x_c=-16.2775
  y_c=-7.679
  z_c=-21.4045
  x_s=34.3309
  y_s=30.258
  z_s=42.4485
  
  echo $x_c>Vina_params/$param_file
  echo $y_c>>Vina_params/$param_file
  echo $z_c>>Vina_params/$param_file
  echo $x_s>>Vina_params/$param_file
  echo $y_s>>Vina_params/$param_file
  echo $z_s>>Vina_params/$param_file

fi

# Initialize the variables

xc=""
yc=""
zc=""
xs=""
ys=""
zs=""

# Read the content of the file into variables using a loop
while IFS= read -r line; do
  if [ -z "$xc" ]; then
    xc="$line"
  elif [ -z "$yc" ]; then
    yc="$line"
  elif [ -z "$zc" ]; then
    zc="$line"
  elif [ -z "$xs" ]; then
    xs="$line"
  elif [ -z "$ys" ]; then
    ys="$line"
  elif [ -z "$zs" ]; then
    zs="$line"
  fi
done < Vina_params/"$param_file"





vina --receptor PDBQT_files/$1.pdbqt --ligand PDBQT_files/$2.pdbqt --center_x $xc --center_y $yc --center_z $zc --size_x $xs --size_y $ys --size_z $zs --out Vina_results/$1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V